### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark
!pip install pyspark
!pip install pyspark2pmml
!pip install py4j 
!pip install findspark

     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 48.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=548d8832bba8990c3c92d0c7d48c28784d5394513e9270bf6b9020be86db77e7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Created wheel for pyspark2pmml: filename=pyspark2pmml-0.5.1-py3-none-any.whl size=2418 sha256=3056485987015e765a51075252e5f12838137b9f02a1683fb04f22dae0eb33b3
  Stored in directory: /root/.cache/pip/wheels/86/31/a6/934cee5fbba108e21ebabccb4b79b088cf550455e396447790
Successfully built pyspark2pmml


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2022-03-06 12:10:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.02s   

2022-03-06 12:10:14 (14.1 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [ ]:
# Load the csv into a spark dataframe
data_dir = "searchterms.csv"
df = spark.read.csv(data_dir, header=True)
df.show()
#from pyspark.sql.types import DoubleType
#df = df.withColumn("x", df.x.cast(DoubleType()))
#df = df.withColumn("y", df.y.cast(DoubleType()))
#df = df.withColumn("z", df.z.cast(DoubleType()))

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|        laptop|
| 12|   11|2021|        laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| gaming laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|        laptop|
+---+-----+----+--------------+
only showing top 20 rows



In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg
row = df.count()
   
# extracting number of columns from the Dataframe
col = len(df.columns)

print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

Number of Rows are: 10000
Number of Columns are: 4


In [ ]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [ ]:
df

DataFrame[day: double, month: double, year: double, searchterm: string, features: vector]

In [ ]:
from pyspark.sql.types import DoubleType
df = df.withColumn("day", df.day.cast(DoubleType()))
df = df.withColumn("month", df.month.cast(DoubleType()))
df = df.withColumn("year", df.year.cast(DoubleType()))
#df = df.withColumn("searchterm", df.z.cast(DoubleType()))

In [ ]:
from pyspark.ml.feature import VectorAssembler

numericCols = ['day', 'month', 'year']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show()

+----+-----+------+--------------+------------------+
| day|month|  year|    searchterm|          features|
+----+-----+------+--------------+------------------+
|12.0| 11.0|2021.0| mobile 6 inch|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0| mobile latest|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|   tablet wifi|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|laptop 14 inch|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|     mobile 5g|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0| mobile 6 inch|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|        laptop|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|        laptop|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|     mobile 5g|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|   tablet wifi|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|     mobile 5g|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0| gaming laptop|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|     mobile 5g|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0|     mobile 5g|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0| mobile 6 inch|[12.0,11.0,2021.0]|
|12.0| 11.0|2021.0| mobile l

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
# Create a LR model
lr = LinearRegression(featuresCol='features', labelCol='weight', maxIter=100)

# Fit the model
lrModel = lr.fit(df)

IllegalArgumentException: 'Field "weight" does not exist.\nAvailable fields: day, month, year, searchterm, features'

In [ ]:
from pyspark.ml.feature import StringIndexer

label_stringIdx = StringIndexer(inputCol = 'class', outputCol = 'labelIndex')
df = label_stringIdx.fit(df).transform(df)
df.show()

In [ ]:
train, test = df.randomSplit([0.8, 0.2], seed = 1)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'labelIndex', maxDepth=7, numTrees=20)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('x', 'y', 'z', 'labelIndex', 'rawPrediction', 'prediction', 'probability').show(25)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [ ]:
df.head(5)

[Row(day=12.0, month=11.0, year=2021.0, searchterm='mobile 6 inch'),
 Row(day=12.0, month=11.0, year=2021.0, searchterm='mobile latest'),
 Row(day=12.0, month=11.0, year=2021.0, searchterm='tablet wifi'),
 Row(day=12.0, month=11.0, year=2021.0, searchterm='laptop 14 inch'),
 Row(day=12.0, month=11.0, year=2021.0, searchterm='mobile 5g')]

In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [ ]:
df.schema[3]

StructField(searchterm,StringType,true)

In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [ ]:
print("The number of times gaming laptop appears in the dataframe: {}".format(df[df['searchterm']== 'gaming laptop'].count()))

The number of times gaming laptop appears in the dataframe: 499


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [ ]:
df.groupBy('searchterm').count().orderBy('count', ascending=False).show(5)

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.gzip

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.gzip

--2022-03-06 12:10:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.gzip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722 (1.7K) [application/gzip]
Saving to: ‘model.gzip’

model.gzip          100%[===================>]   1.68K  --.-KB/s    in 0s      

2022-03-06 12:10:30 (304 MB/s) - ‘model.gzip’ saved [1722/1722]



In [ ]:
!tar -zxvf model.gzip

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-f37d8b09-cd1a-426c-ba90-4047208b011b-c000.snappy.parquet
sales_prediction.model/data/.part-00000-f37d8b09-cd1a-426c-ba90-4047208b011b-c000.snappy.parquet.crc
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/._SUCCESS.crc


In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [ ]:
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [ ]:
# This function converts a scalar number into a dataframe that can be used by the model to predict.
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [ ]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|176.14285712605306|
+------------------+

